# 04. 結果分析・可視化

このノートブックでは、評価結果の詳細分析と可視化を行います。

## 処理内容
1. 評価結果の読み込み
2. 予測精度の可視化
3. 誤差分布の分析
4. ケース別性能分析
5. モデル比較（複数モデルがある場合）
6. インタラクティブダッシュボード作成
7. 報告書生成

In [ ]:
# 必要なライブラリのインポート
import sys
from pathlib import Path

# プロジェクトルートをパスに追加
project_root = Path.cwd().parent
sys.path.append(str(project_root))

from config.experiment_configs import (
    get_baseline_config, get_cvae_config, get_gnn_config,
    get_large_model_config
)
from src.visualization.plotting import RIMDVisualizer, create_visualization_report
from src.utils.experiment_manager import ExperimentManager
import logging
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, HTML

# 表示設定
%matplotlib inline
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# ログレベル設定
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## 分析対象の選択

分析したい実験を選択してください：

In [ ]:
# ===========================================
# 分析対象実験選択
# ===========================================

# 1. ベースライン実験分析
config = get_baseline_config()

# 2. CVAE実験分析
# config = get_cvae_config()

# 3. GNN実験分析
# config = get_gnn_config()

# 4. 大規模モデル実験分析
# config = get_large_model_config()

# ===========================================
# カスタム分析設定
# ===========================================
# config = get_baseline_config()
# config.exp_id = "custom_experiment"  # 分析したい実験IDに変更

print(f"分析対象実験: {config.exp_id}")
print(f"説明: {config.description}")
print(f"モデルタイプ: {config.model.model_type}")

In [ ]:
# 実験マネージャーの作成
exp_manager = ExperimentManager(config)

# 評価結果の存在確認
eval_dir = exp_manager.exp_dir / "evaluation"
predictions_dir = exp_manager.exp_dir / "predictions"

if not eval_dir.exists() or not (eval_dir / "detailed_results.json").exists():
    print("❌ 評価結果が見つかりません。03_evaluation.ipynb を先に実行してください。")
    raise FileNotFoundError("No evaluation results found")

print(f"✅ 評価結果ディレクトリ: {eval_dir}")
print(f"✅ 予測結果ディレクトリ: {predictions_dir}")

In [ ]:
# 評価結果の読み込み
with open(eval_dir / "detailed_results.json", 'r', encoding='utf-8') as f:
    detailed_results = json.load(f)

with open(eval_dir / "evaluation_summary.json", 'r', encoding='utf-8') as f:
    evaluation_summary = json.load(f)

print("✅ 評価結果読み込み完了")
print(f"分析対象データ分割: {list(detailed_results.keys())}")

In [ ]:
# 予測結果データの読み込み
predictions_data = {}

for split in detailed_results.keys():
    pred_file = predictions_dir / f"{split}_predictions.npy"
    target_file = predictions_dir / f"{split}_targets.npy"
    csv_file = predictions_dir / f"{split}_predictions.csv"
    
    if pred_file.exists() and target_file.exists():
        predictions_data[split] = {
            'predictions': np.load(pred_file),
            'targets': np.load(target_file)
        }
        
        # CSVファイルがある場合はケースIDも読み込み
        if csv_file.exists():
            csv_data = pd.read_csv(csv_file)
            predictions_data[split]['case_ids'] = csv_data['case_id'].tolist()
        else:
            predictions_data[split]['case_ids'] = []

print(f"✅ 予測結果読み込み完了: {list(predictions_data.keys())}")

## 基本的な可視化

予測精度と誤差分布の基本的な可視化を行います。

In [ ]:
# 可視化ディレクトリの作成
viz_dir = exp_manager.exp_dir / "visualizations"
viz_dir.mkdir(exist_ok=True)

# 可視化器の作成
visualizer = RIMDVisualizer(viz_dir)
print(f"✅ 可視化器作成完了: {viz_dir}")

In [ ]:
# テストセットの予測精度プロット
if 'test' in predictions_data:
    test_data = predictions_data['test']
    
    print("=== テストセット予測精度 ===")
    fig = visualizer.plot_prediction_accuracy(
        test_data['predictions'],
        test_data['targets'],
        test_data['case_ids'],
        f"Test Set Prediction Accuracy - {config.exp_id}"
    )
    plt.show()
    
    # 基本統計表示
    test_metrics = detailed_results['test']['metrics']
    print(f"RMSE: {test_metrics['rmse_mm']:.3f} mm")
    print(f"MAE: {test_metrics['mae_mm']:.3f} mm")
    print(f"相関係数 (X): {np.corrcoef(test_data['predictions'][:, 0], test_data['targets'][:, 0])[0, 1]:.4f}")
    print(f"相関係数 (Y): {np.corrcoef(test_data['predictions'][:, 1], test_data['targets'][:, 1])[0, 1]:.4f}")

In [ ]:
# 誤差分布の詳細分析
if 'test' in predictions_data:
    print("\n=== 誤差分布分析 ===")
    fig = visualizer.plot_error_distribution(
        test_data['predictions'],
        test_data['targets'],
        f"Error Distribution Analysis - {config.exp_id}"
    )
    plt.show()
    
    # 統計サマリー
    errors = test_data['predictions'] - test_data['targets']
    euclidean_errors = np.linalg.norm(errors, axis=1)
    
    print(f"ユークリッド誤差統計:")
    print(f"  平均: {np.mean(euclidean_errors):.3f} mm")
    print(f"  標準偏差: {np.std(euclidean_errors):.3f} mm")
    print(f"  中央値: {np.median(euclidean_errors):.3f} mm")
    print(f"  最大値: {np.max(euclidean_errors):.3f} mm")
    print(f"  P95: {np.percentile(euclidean_errors, 95):.3f} mm")
    print(f"  P99: {np.percentile(euclidean_errors, 99):.3f} mm")

In [ ]:
# ケース別性能比較（ケースIDがある場合）
if 'test' in predictions_data and predictions_data['test']['case_ids']:
    print("\n=== ケース別性能比較 ===")
    fig = visualizer.plot_case_comparison(
        test_data['predictions'],
        test_data['targets'],
        test_data['case_ids'],
        max_cases=10
    )
    if fig:
        plt.show()
        
        # ケース統計の表示
        case_analysis = detailed_results['test'].get('case_analysis', {})
        if case_analysis and '_summary' in case_analysis:
            summary = case_analysis['_summary']
            print(f"最良ケース: {summary['best_case']}")
            print(f"最悪ケース: {summary['worst_case']}")
            variability = summary['case_variability']
            print(f"ケース間変動: {variability['mean_error_std']:.3f} mm (標準偏差)")
else:
    print("\nケースIDが利用できないため、ケース別分析をスキップします。")

## 学習曲線の分析

学習過程の可視化を行います。

In [ ]:
# 学習ログの読み込み
metrics_log_path = exp_manager.exp_dir / "logs" / "metrics.jsonl"

if metrics_log_path.exists():
    print("=== 学習曲線分析 ===")
    
    # メトリクス履歴の読み込み
    metrics_history = {'train': [], 'val': []}
    
    with open(metrics_log_path, 'r') as f:
        for line in f:
            log_entry = json.loads(line.strip())
            epoch = log_entry.get('epoch', 0)
            
            if 'train_metrics' in log_entry:
                train_metrics = log_entry['train_metrics']
                train_metrics['epoch'] = epoch
                metrics_history['train'].append(train_metrics)
            
            if 'val_metrics' in log_entry:
                val_metrics = log_entry['val_metrics']
                val_metrics['epoch'] = epoch
                metrics_history['val'].append(val_metrics)
    
    # 学習曲線プロット
    if metrics_history['train'] or metrics_history['val']:
        fig = visualizer.plot_learning_curves(
            metrics_history,
            f"Learning Curves - {config.exp_id}"
        )
        plt.show()
        
        # 学習統計
        if metrics_history['val']:
            final_val_metrics = metrics_history['val'][-1]
            print(f"最終エポック: {final_val_metrics.get('epoch', 'N/A')}")
            print(f"最終検証損失: {final_val_metrics.get('total', final_val_metrics.get('reconstruction', 'N/A'))}")
            if 'rmse_mm' in final_val_metrics:
                print(f"最終検証RMSE: {final_val_metrics['rmse_mm']:.3f} mm")
    else:
        print("学習履歴データが見つかりません。")
else:
    print("学習ログファイルが見つかりません。")

## インタラクティブ可視化

Plotlyを使用したインタラクティブな可視化を作成します。

In [ ]:
# インタラクティブ誤差プロット
if 'test' in predictions_data:
    print("=== インタラクティブ誤差分析 ===")
    
    interactive_fig = visualizer.create_interactive_error_plot(
        test_data['predictions'],
        test_data['targets'],
        test_data['case_ids'] if test_data['case_ids'] else None
    )
    
    # Jupyter環境で表示
    interactive_fig.show()
    
    print(f"インタラクティブプロットが {viz_dir / 'interactive_error_plot.html'} に保存されました。")

In [ ]:
# 予測vs実測の3Dプロット
if 'test' in predictions_data:
    print("\n=== 3D予測精度可視化 ===")
    
    test_pred = test_data['predictions']
    test_target = test_data['targets']
    euclidean_errors = np.linalg.norm(test_pred - test_target, axis=1)
    
    # 3Dプロット作成
    fig_3d = go.Figure(data=[
        go.Scatter3d(
            x=test_target[:, 0],
            y=test_target[:, 1],
            z=euclidean_errors,
            mode='markers',
            marker=dict(
                size=4,
                color=euclidean_errors,
                colorscale='Viridis',
                colorbar=dict(title="Error (mm)"),
                opacity=0.7
            ),
            text=[f'Case: {cid}' for cid in test_data['case_ids']] if test_data['case_ids'] else None,
            hovertemplate='Target X: %{x:.3f}<br>Target Y: %{y:.3f}<br>Error: %{z:.3f}<br>%{text}<extra></extra>'
        )
    ])
    
    fig_3d.update_layout(
        title=f"3D Error Distribution - {config.exp_id}",
        scene=dict(
            xaxis_title="Target X Displacement (mm)",
            yaxis_title="Target Y Displacement (mm)",
            zaxis_title="Prediction Error (mm)"
        ),
        width=800,
        height=600
    )
    
    fig_3d.show()
    fig_3d.write_html(str(viz_dir / "3d_error_plot.html"))

## モデル比較分析

複数のモデルがある場合の比較分析を行います。

In [ ]:
# モデル比較結果の読み込み
comparison_dir = exp_manager.exp_dir / "model_comparison"
comparison_file = comparison_dir / "comparison_results.json"

if comparison_file.exists():
    print("=== モデル比較分析 ===")
    
    with open(comparison_file, 'r', encoding='utf-8') as f:
        comparison_data = json.load(f)
    
    comparison_results = comparison_data['results']
    comparison_analysis = comparison_data['comparison']
    
    # 比較可視化
    if len(comparison_results) > 1:
        fig = visualizer.plot_model_comparison(
            comparison_results,
            "Model Performance Comparison"
        )
        plt.show()
        
        # 最良モデル表示
        print("\n最良モデル:")
        for metric, best in comparison_analysis['best_models'].items():
            print(f"  {metric}: {best['model']} ({best['value']:.3f})")
        
        # インタラクティブ比較表作成
        models = list(comparison_results.keys())
        metrics = ['rmse_mm', 'mae_mm', 'median_error_mm', 'p95_error_mm']
        
        comparison_df = pd.DataFrame(comparison_results).T
        comparison_df = comparison_df[metrics]
        comparison_df.columns = ['RMSE (mm)', 'MAE (mm)', 'Median Error (mm)', 'P95 Error (mm)']
        
        print("\n性能比較表:")
        display(comparison_df.round(3))
        
        # レーダーチャート作成
        fig_radar = go.Figure()
        
        for model_name in models:
            model_metrics = comparison_results[model_name]
            
            # 正規化（小さいほど良いので反転）
            normalized_values = []
            metric_names = []
            
            for metric in metrics:
                if metric in model_metrics:
                    # 最大値で正規化して反転
                    max_val = max([comparison_results[m].get(metric, 0) for m in models])
                    normalized_val = (max_val - model_metrics[metric]) / max_val if max_val > 0 else 0
                    normalized_values.append(normalized_val)
                    metric_names.append(metric.replace('_mm', '').upper())
            
            fig_radar.add_trace(go.Scatterpolar(
                r=normalized_values,
                theta=metric_names,
                fill='toself',
                name=model_name
            ))
        
        fig_radar.update_layout(
            polar=dict(
                radialaxis=dict(
                    visible=True,
                    range=[0, 1]
                )
            ),
            title="Model Performance Radar Chart",
            showlegend=True
        )
        
        fig_radar.show()
        fig_radar.write_html(str(viz_dir / "model_comparison_radar.html"))
        
else:
    print("モデル比較結果が見つかりません。複数モデルの評価を実行していない可能性があります。")

## 統計的有意性検定

予測精度の統計的分析を行います。

In [ ]:
# 統計的分析
if 'test' in predictions_data:
    print("=== 統計的分析 ===")
    
    from scipy import stats
    import scipy.stats as stats_scipy
    
    test_pred = test_data['predictions']
    test_target = test_data['targets']
    errors = test_pred - test_target
    
    # 残差の正規性検定
    print("\n残差の正規性検定 (Shapiro-Wilk):")
    for i, coord in enumerate(['X', 'Y']):
        stat, p_value = stats.shapiro(errors[:, i])
        print(f"{coord}方向: W={stat:.4f}, p={p_value:.6f}")
        if p_value > 0.05:
            print(f"  → {coord}方向の残差は正規分布に従う (p > 0.05)")
        else:
            print(f"  → {coord}方向の残差は正規分布に従わない (p ≤ 0.05)")
    
    # 残差の等分散性検定（Levene検定）
    print("\n残差の等分散性:")
    x_errors = errors[:, 0]
    y_errors = errors[:, 1]
    
    # グループ分け（予測値の大きさで）
    x_median = np.median(test_pred[:, 0])
    y_median = np.median(test_pred[:, 1])
    
    x_low_group = x_errors[test_pred[:, 0] <= x_median]
    x_high_group = x_errors[test_pred[:, 0] > x_median]
    y_low_group = y_errors[test_pred[:, 1] <= y_median]
    y_high_group = y_errors[test_pred[:, 1] > y_median]
    
    x_levene_stat, x_levene_p = stats.levene(x_low_group, x_high_group)
    y_levene_stat, y_levene_p = stats.levene(y_low_group, y_high_group)
    
    print(f"X方向 Levene統計量: {x_levene_stat:.4f}, p={x_levene_p:.6f}")
    print(f"Y方向 Levene統計量: {y_levene_stat:.4f}, p={y_levene_p:.6f}")
    
    # 予測値と実測値の相関
    print("\n予測値-実測値相関:")
    for i, coord in enumerate(['X', 'Y']):
        r, p_value = stats.pearsonr(test_pred[:, i], test_target[:, i])
        print(f"{coord}方向: r={r:.4f}, p={p_value:.6f}")
        print(f"  決定係数 R²: {r**2:.4f}")
    
    # 平均誤差のt検定（0との比較）
    print("\n平均誤差の有意性検定 (one-sample t-test):")
    for i, coord in enumerate(['X', 'Y']):
        t_stat, p_value = stats.ttest_1samp(errors[:, i], 0)
        print(f"{coord}方向: t={t_stat:.4f}, p={p_value:.6f}")
        if abs(p_value) > 0.05:
            print(f"  → {coord}方向に系統的バイアスはない (p > 0.05)")
        else:
            print(f"  → {coord}方向に系統的バイアスあり (p ≤ 0.05)")

## 予測不確実性分析（CVAEの場合）

CVAEモデルの場合の不確実性分析を行います。

In [ ]:
# CVAE不確実性分析
if config.model.use_cvae and 'test' in predictions_data:
    print("=== CVAE不確実性分析 ===")
    
    # 複数回サンプリングによる不確実性推定
    # （実際の実装では、モデルから複数回サンプリングする必要があります）
    
    # ここではデモ用に誤差の分散を不確実性として扱います
    euclidean_errors = np.linalg.norm(test_pred - test_target, axis=1)
    
    # 不確実性の可視化
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 誤差vs予測値の散布図
    axes[0, 0].scatter(np.linalg.norm(test_pred, axis=1), euclidean_errors, alpha=0.6)
    axes[0, 0].set_xlabel('Prediction Magnitude (mm)')
    axes[0, 0].set_ylabel('Prediction Error (mm)')
    axes[0, 0].set_title('Error vs Prediction Magnitude')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 不確実性ヒストグラム
    axes[0, 1].hist(euclidean_errors, bins=50, alpha=0.7, edgecolor='black')
    axes[0, 1].set_xlabel('Prediction Error (mm)')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].set_title('Prediction Uncertainty Distribution')
    axes[0, 1].grid(True, alpha=0.3)
    
    # 信頼区間分析
    confidence_levels = [50, 68, 95, 99]
    percentiles = np.percentile(euclidean_errors, confidence_levels)
    
    axes[1, 0].bar(confidence_levels, percentiles, alpha=0.7)
    axes[1, 0].set_xlabel('Confidence Level (%)')
    axes[1, 0].set_ylabel('Error Threshold (mm)')
    axes[1, 0].set_title('Confidence Intervals')
    axes[1, 0].grid(True, alpha=0.3)
    
    # 不確実性vs実誤差の較正プロット
    # （実際の実装では、推定不確実性と実誤差の関係をプロット）
    sorted_indices = np.argsort(euclidean_errors)
    sorted_errors = euclidean_errors[sorted_indices]
    axes[1, 1].plot(np.arange(len(sorted_errors)), sorted_errors, 'b-', linewidth=2)
    axes[1, 1].set_xlabel('Sample Index (sorted by error)')
    axes[1, 1].set_ylabel('Prediction Error (mm)')
    axes[1, 1].set_title('Error Distribution (Sorted)')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.suptitle(f'CVAE Uncertainty Analysis - {config.exp_id}', fontsize=16)
    plt.tight_layout()
    plt.savefig(viz_dir / "cvae_uncertainty_analysis.png", dpi=300, bbox_inches='tight')
    plt.show()
    
    # 不確実性統計
    print(f"\n不確実性統計:")
    print(f"  平均誤差: {np.mean(euclidean_errors):.3f} mm")
    print(f"  誤差標準偏差: {np.std(euclidean_errors):.3f} mm")
    for i, level in enumerate(confidence_levels):
        print(f"  {level}%信頼区間: {percentiles[i]:.3f} mm")
        
else:
    print("CVAEモデルではないため、不確実性分析をスキップします。")

## 包括的レポート生成

分析結果をまとめた包括的なレポートを生成します。

In [ ]:
# 包括的可視化レポート作成
print("=== 包括的可視化レポート作成 ===")

try:
    create_visualization_report(predictions_data, viz_dir)
    print(f"✅ 可視化レポート作成完了: {viz_dir}")
except Exception as e:
    print(f"❌ 可視化レポート作成エラー: {e}")

In [ ]:
# 最終分析サマリーレポート作成
report_content = []
report_content.append(f"# RIMD Model Analysis Report - {config.exp_id}\n")
report_content.append(f"**実験説明**: {config.description}\n")
report_content.append(f"**モデルタイプ**: {config.model.model_type}\n")
report_content.append(f"**CVAE使用**: {config.model.use_cvae}\n")
report_content.append(f"**分析日時**: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")

# 性能サマリー
if 'test' in detailed_results:
    test_metrics = detailed_results['test']['metrics']
    report_content.append("## 全体性能\n")
    report_content.append(f"- **RMSE**: {test_metrics['rmse_mm']:.3f} mm\n")
    report_content.append(f"- **MAE**: {test_metrics['mae_mm']:.3f} mm\n")
    report_content.append(f"- **Median Error**: {test_metrics['median_error_mm']:.3f} mm\n")
    report_content.append(f"- **P95 Error**: {test_metrics['p95_error_mm']:.3f} mm\n")
    report_content.append(f"- **Max Error**: {test_metrics['max_error_mm']:.3f} mm\n")
    if test_metrics.get('gain_percent'):
        report_content.append(f"- **Improvement Gain**: {test_metrics['gain_percent']:.1f}%\n")
    report_content.append("\n")

# 主要発見事項
report_content.append("## 主要発見事項\n")
for finding in evaluation_summary.get('key_findings', []):
    report_content.append(f"- {finding}\n")
report_content.append("\n")

# 生成されたファイル一覧
report_content.append("## 生成されたファイル\n")
report_content.append("### 可視化ファイル\n")
for viz_file in viz_dir.glob("*.png"):
    report_content.append(f"- {viz_file.name}\n")
for viz_file in viz_dir.glob("*.html"):
    report_content.append(f"- {viz_file.name} (インタラクティブ)\n")
report_content.append("\n")

# レポート保存
report_path = viz_dir / "analysis_summary_report.md"
with open(report_path, 'w', encoding='utf-8') as f:
    f.write(''.join(report_content))

print(f"✅ 分析サマリーレポート作成: {report_path}")

# HTMLレポート作成
html_content = ''.join(report_content).replace('\n', '<br>')
html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <title>RIMD Analysis Report - {config.exp_id}</title>
    <style>
        body {{ font-family: Arial, sans-serif; margin: 40px; }}
        h1, h2 {{ color: #333; }}
        .metric {{ background-color: #f5f5f5; padding: 10px; margin: 5px 0; border-radius: 5px; }}
    </style>
</head>
<body>
    {html_content}
</body>
</html>
"""

html_report_path = viz_dir / "analysis_summary_report.html"
with open(html_report_path, 'w', encoding='utf-8') as f:
    f.write(html_content)

print(f"✅ HTML分析レポート作成: {html_report_path}")

## 分析完了

結果分析・可視化が完了しました。

### 生成されたファイル
- ✅ 静的プロット（PNG形式）
- ✅ インタラクティブプロット（HTML形式）
- ✅ 分析サマリーレポート（Markdown & HTML）
- ✅ 統計分析結果
- ✅ モデル比較結果（複数モデルがある場合）

### 次のステップ
1. 生成されたレポートとプロットの確認
2. 性能改善のためのハイパーパラメータ調整
3. 異なるモデル構成での追加実験
4. 結果の論文・レポート化

### ファイル場所
すべての分析結果は以下に保存されています：

In [ ]:
# 最終ファイル一覧表示
print("=== 生成されたファイル一覧 ===")
print(f"\n実験ディレクトリ: {exp_manager.exp_dir}")
print(f"\n可視化ファイル: {viz_dir}")
for file_path in sorted(viz_dir.glob("*")):
    if file_path.is_file():
        print(f"  - {file_path.name}")

print(f"\n評価結果: {eval_dir}")
for file_path in sorted(eval_dir.glob("*")):
    if file_path.is_file():
        print(f"  - {file_path.name}")

print(f"\n予測結果: {predictions_dir}")
if predictions_dir.exists():
    for file_path in sorted(predictions_dir.glob("*")):
        if file_path.is_file():
            print(f"  - {file_path.name}")

print("\n✅ 分析・可視化完了")
print("\n📊 インタラクティブプロットはHTMLファイルをブラウザで開いて確認してください")
print("📋 詳細な分析結果はMarkdown/HTMLレポートをご確認ください")